## Feature Scaling and Normalization - Lab

## Introduction
In this lab, you'll practice your feature scaling and normalization skills!

## Objectives
You will be able to:
* Implement min-max scaling, mean-normalization, log normalization and unit vector normalization in python
* Identify appropriate normalization and scaling techniques for given dataset

## Back to our Boston Housing data

Let's import our Boston Housing data. Remember we categorized two variables and deleted the "NOX" (nitride oxide concentration) variable because it was highly correlated with two other features.

In [1]:
import pandas as pd
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)

# first, create bins for based on the values observed. 5 values will result in 4 bins
bins = [0, 3, 4 , 5, 24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 5 values will result in 4 bins
bins = [0, 250, 300, 360, 460, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)
boston_features = boston_features.drop("NOX",axis=1)

## Look at the histograms for the continuous variables

In [2]:
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
boston_features.loc[:, ['AGE', 'B', 'CRIM', 'DIS', 'INDUS', 'LSTAT', 'PTRATIO', 'ZN']].hist(figsize = (25, 15), color = 'b')

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a16ea6588>,
      dtype=object)

## Perform log transformations for the variables where it makes sense

Analyze the results in terms of how they improved the normality performance. What is the problem with the "ZN" variable?  

In [3]:
boston_features.head()

,CRIM,ZN,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 3]","RAD_(3, 4]","RAD_(4, 5]","RAD_(5, 24]","TAX_(0, 250]","TAX_(250, 300]","TAX_(300, 360]","TAX_(360, 460]","TAX_(460, 712]"
0,0.00632,18.0,2.31,0.0,6.575,65.2,4.0900,15.3,396.90,4.98,1,0,0,0,0,1,0,0,0
1,0.02731,0.0,7.07,0.0,6.421,78.9,4.9671,17.8,396.90,9.14,1,0,0,0,1,0,0,0,0
2,0.02729,0.0,7.07,0.0,7.185,61.1,4.9671,17.8,392.83,4.03,1,0,0,0,1,0,0,0,0
3,0.03237,0.0,2.18,0.0,6.998,45.8,6.0622,18.7,394.63,2.94,1,0,0,0,1,0,0,0,0
4,0.06905,0.0,2.18,0.0,7.147,54.2,6.0622,18.7,396.90,5.33,1,0,0,0,1,0,0,0,0


In [4]:
import numpy as np
data_log = pd.DataFrame([])
# data_log['logage'] = np.log(boston_features['AGE'])
# data_log['logB'] = np.log(boston_features['B'])
data_log['logcrim'] = np.log(boston_features['CRIM'])
data_log['logdis'] = np.log(boston_features['DIS'])
data_log['logindus'] = np.log(boston_features['INDUS'])
data_log['logLSTAT'] = np.log(boston_features['LSTAT'])
data_log['logpt'] = np.log(boston_features['PTRATIO'])
# data_log['logzn'] = np.log(boston_features['ZN'])
data_log.hist(figsize  = [25, 25])

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a1a6c8a90>,
      dtype=object)

"ZN" has a lot of zeros (more than 50%!). Remember that this variable denoted: "proportion of residential land zoned for lots over 25,000 sq.ft.". It might have made sense to categorize this variable to "over 25,000 feet or not (binary variable 1/0). Now you have a zero-inflated variable which is cumbersome to work with.

## Try different types of transformations on the continuous variables

Store your final features in a dataframe `features_final`

In [5]:
logcrim = data_log['logcrim']
logdis = data_log['logdis']
logindus = data_log['logindus']
loglstat = data_log['logLSTAT']
logpt = data_log['logpt']

scaled_crim = (logcrim - np.mean(logcrim)) / np.sqrt(np.var(logcrim))
scaled_dis = (logdis - min(logdis))/(max(logdis) - min(logdis))
scaled_indus = (logindus - np.mean(logindus))/(max(logindus) - min(logindus))
scaled_lstat = (loglstat - np.mean(loglstat)) / np.sqrt(np.var(loglstat))
scaled_pt = (logpt - min(logpt))/(max(logpt) - min(logpt))


features_final = pd.DataFrame([])
features_final['over_25k'] = list(map(lambda row: 1 if row > 0 else 0, boston_features['ZN']))
features_final['scaled_crim'] = logcrim
features_final['scaled_dis'] = scaled_dis
features_final['scaled_indus'] = scaled_indus
features_final['scaled_lstat'] = scaled_lstat
features_final['scaled_pt'] = scaled_pt
features_final = pd.concat([features_final, boston_features.iloc[:, 10:]], axis=1)
features_final.head()

,over_25k,scaled_crim,scaled_dis,scaled_indus,scaled_lstat,scaled_pt,"RAD_(0, 3]","RAD_(3, 4]","RAD_(4, 5]","RAD_(5, 24]","TAX_(0, 250]","TAX_(250, 300]","TAX_(300, 360]","TAX_(360, 460]","TAX_(460, 712]"
0,1,-5.064036,0.542096,-0.322716,-1.275260,0.348358,1,0,0,0,0,1,0,0,0
1,0,-3.600502,0.623954,-0.049844,-0.263711,0.619906,1,0,0,0,1,0,0,0,0
2,0,-3.601235,0.623954,-0.049844,-1.627858,0.619906,1,0,0,0,1,0,0,0,0
3,0,-3.430523,0.707895,-0.336846,-2.153192,0.708405,1,0,0,0,1,0,0,0,0
4,0,-2.672924,0.707895,-0.336846,-1.162114,0.708405,1,0,0,0,1,0,0,0,0


## Summary
Great! You've now transformed your final data using feature scaling and normalization, and stored them in the `features_final` dataframe.